# House Prices - Preprocessing Data


## Preparation

### Import necessary libraries

In [1]:
from os.path import join
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

### Load train.csv file

In [2]:
data_dir = join('..', '..', 'data')
input_dir = join(data_dir, 'raw')
output_dir = join(data_dir, 'preprocessed')

house = pd.read_csv(join(input_dir, "train.csv"))
df_train = house.copy()
test_dataset = house = pd.read_csv(join(input_dir, "test.csv"))
df_test = house.copy()

print("Successfully load training data.")
df_train.head()

print("Successfully saved preprocessed data.")

Successfully load training data.
Successfully saved preprocessed data.


In [3]:
print("===== Number of missing values (train dataset) =====\n")
print(df_train.isnull().sum()[df_train.isnull().sum() > 0])

print('\n')

print("===== Number of missing values (test dataset) =====\n")
print(df_test.isnull().sum()[df_test.isnull().sum() > 0])


===== Number of missing values (train dataset) =====

LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


===== Number of missing values (test dataset) =====

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType       894
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType       

Tập train và test có dùng số lượng dữ liệu missing 

## Xử lý dữ liệu bị thiếu

### Xử lý biến định tuyến

Danh sách none_cols chứa các categorical features (kiểu object/string) mà:

- NaN sẽ chuyển đổi về 'NA' hoặc 'None' dựa trên thông tin mô tả của tập dữ liệu, vì địa điểm không có feature này.

In [4]:
none_cols = [
    'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'
]

for col in none_cols:
    if col in df_train.columns and col in df_test.columns:
        # df_train[col] = df_train[col].fillna('None')
        df_train[col] = df_train[col].fillna('NA')
        df_test[col] = df_test[col].fillna('NA')
df_train['MasVnrType'] = df_train['MasVnrType'].fillna('None')
df_test['MasVnrType'] = df_test['MasVnrType'].fillna('None')

- Các cột dữ liệu phân loại còn lại: điền mode.

In [5]:
cat_cols_with_missing = df_train.select_dtypes(include=['object']).columns[
    df_train.select_dtypes(include=['object']).isnull().any()
]
for col in cat_cols_with_missing:
    df_train[col] = df_train[col].fillna(df_train[col].mode()[0])
    
cat_cols_with_missing = df_test.select_dtypes(include=['object']).columns[
    df_test.select_dtypes(include=['object']).isnull().any()
]
for col in cat_cols_with_missing:
    df_test[col] = df_test[col].fillna(df_test[col].mode()[0])

### Đặc trưng biểu diễn số liệu

Điền `0` cho các cột số khi thiếu dữ liệu là hợp lý vì:

- Các cột này biểu diễn số lượng/diện tích.

- Thiếu dữ liệu = thực tế không có.


In [6]:
zero_cols = [
    'GarageYrBlt', 'GarageArea', 'GarageCars',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
    'MasVnrArea'
]
for col in zero_cols:
    if col in df_train.columns and col in df_test.columns:
        df_train[col] = df_train[col].fillna(0)
        df_test[col] = df_test[col].fillna(0)


- Các cột số còn lại (hiếm): điền median

In [7]:
num_cols_with_missing = df_train.select_dtypes(include=['int64', 'float64']).columns[
    df_train.select_dtypes(include=['int64', 'float64']).isnull().any()
]
for col in num_cols_with_missing:
    df_train[col] = df_train[col].fillna(df_train[col].median())
    
num_cols_with_missing = df_test.select_dtypes(include=['int64', 'float64']).columns[
    df_test.select_dtypes(include=['int64', 'float64']).isnull().any()
]
for col in num_cols_with_missing:
    df_test[col] = df_test[col].fillna(df_test[col].median())

### Đặc trưng LotFrontage

Là chiều dài mặt tiền lô đất (đơn vị feet) tiếp giáp với đường:
- Lô đất lớn hơn thì mặt tiền rộng hơn
- Lô đất trong khu cao cấp (Neighborhood tốt) thường có mặt tiền rộng hơn

In [8]:
if 'LotFrontage' in df_train.columns:
    df_train['LotFrontage'] = df_train.groupby('Neighborhood')['LotFrontage'].transform(
        lambda x: x.fillna(x.median())
    )
    # Nếu vẫn còn missing (Neighborhood hiếm), điền median toàn cục
    df_train['LotFrontage'] = df_train['LotFrontage'].fillna(df_train['LotFrontage'].median())
    
if 'LotFrontage' in df_test.columns:
    df_test['LotFrontage'] = df_test.groupby('Neighborhood')['LotFrontage'].transform(
        lambda x: x.fillna(x.median())
    )
    # Nếu vẫn còn missing (Neighborhood hiếm), điền median toàn cục
    df_test['LotFrontage'] = df_test['LotFrontage'].fillna(df_test['LotFrontage'].median())

In [9]:
# Hàm kiểm tra missing
def missing_report(df_train, name="DataFrame"):
    missing = df_train.isnull().sum()
    missing_percent = (missing / len(df_train)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_percent
    }).sort_values(by='Missing Count', ascending=False)
    missing_df = missing_df[missing_df['Missing Count'] > 0]
    print(f"\n=== Missing Values in {name} ===")
    print(missing_df)
    return missing_df

missing_train = missing_report(df_train, "Train")
missing_test = missing_report(df_test, "Test")


=== Missing Values in Train ===
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []

=== Missing Values in Test ===
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []


## Xử lý dữ liệu ngoài lề

Dựa vào những phân tích ở phần EDA, ta sẽ tiến hành scale lại dữ liệu bằng cách dùng hàm log (log transformation) cho các biến có độ lệch (skewness) lớn hơn hoặc bằng 2.

In [10]:
numerical_columns = df_train.select_dtypes(include=["int64", "float64"]).columns

def detect_outliers_iqr(df_train, col, factor=1.5):
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = max(Q1 - factor * IQR, 0)  # Age cannot be negative
    upper_bound = Q3 + factor * IQR
    outliers = df_train[(df_train[col] < lower_bound) | (df_train[col] > upper_bound)]
    # print(f'{col} - Outliers (IQR): {len(outliers)}, Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f}')
    return outliers, lower_bound, upper_bound


for col in numerical_columns:
    outliers, lower_bound, upper_bound = detect_outliers_iqr(df_train, col)
    print(f'Total outliers in {col}: {len(outliers)}')

Total outliers in Id: 0
Total outliers in MSSubClass: 103
Total outliers in LotFrontage: 106
Total outliers in LotArea: 69
Total outliers in OverallQual: 2
Total outliers in OverallCond: 125
Total outliers in YearBuilt: 7
Total outliers in YearRemodAdd: 0
Total outliers in MasVnrArea: 98
Total outliers in BsmtFinSF1: 7
Total outliers in BsmtFinSF2: 167
Total outliers in BsmtUnfSF: 29
Total outliers in TotalBsmtSF: 61
Total outliers in 1stFlrSF: 20
Total outliers in 2ndFlrSF: 2
Total outliers in LowQualFinSF: 26
Total outliers in GrLivArea: 31
Total outliers in BsmtFullBath: 1
Total outliers in BsmtHalfBath: 82
Total outliers in FullBath: 0
Total outliers in HalfBath: 0
Total outliers in BedroomAbvGr: 35
Total outliers in KitchenAbvGr: 68
Total outliers in TotRmsAbvGrd: 30
Total outliers in Fireplaces: 5
Total outliers in GarageYrBlt: 81
Total outliers in GarageCars: 5
Total outliers in GarageArea: 21
Total outliers in WoodDeckSF: 32
Total outliers in OpenPorchSF: 77
Total outliers in E

In [11]:
for col in numerical_columns:
    if df_train[col].skew() >= 2:
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])

In [12]:
for col in numerical_columns:
    outliers, lower_bound, upper_bound = detect_outliers_iqr(df_train, col)
    print(f'Total outliers in {col}: {len(outliers)}')

Total outliers in Id: 0
Total outliers in MSSubClass: 103
Total outliers in LotFrontage: 123
Total outliers in LotArea: 132
Total outliers in OverallQual: 2
Total outliers in OverallCond: 125
Total outliers in YearBuilt: 7
Total outliers in YearRemodAdd: 0
Total outliers in MasVnrArea: 0
Total outliers in BsmtFinSF1: 7
Total outliers in BsmtFinSF2: 167
Total outliers in BsmtUnfSF: 29
Total outliers in TotalBsmtSF: 61
Total outliers in 1stFlrSF: 20
Total outliers in 2ndFlrSF: 2
Total outliers in LowQualFinSF: 26
Total outliers in GrLivArea: 31
Total outliers in BsmtFullBath: 1
Total outliers in BsmtHalfBath: 82
Total outliers in FullBath: 0
Total outliers in HalfBath: 0
Total outliers in BedroomAbvGr: 35
Total outliers in KitchenAbvGr: 68
Total outliers in TotRmsAbvGrd: 30
Total outliers in Fireplaces: 5
Total outliers in GarageYrBlt: 81
Total outliers in GarageCars: 5
Total outliers in GarageArea: 21
Total outliers in WoodDeckSF: 32
Total outliers in OpenPorchSF: 0
Total outliers in En

Ta có thể thấy rằng có vài đặc trưng lại có số liệu outliers tăng lên sau khi thực hiện scale khoảng dữ liệu bằng hàm log. Điều đó là chấp nhận được bởi vì log transformation không đảm bảo việc loại bỏ hoàn toàn outliers, mà chỉ đảm bảo được việc scale dữ liệu cho sự phân phối dữ liệu đỡ bị lệch. 

Có thể việc scale dữ liệu bằng hàm log làm cho các dữ liệu tập trung vào khoảng giữa của dữ liệu quá nhiều, làm cho các con số chỉ bị lệch đôi chút cũng có thể bị coi như là outliers. 

Để đảm bảo ta không bỏ sót những dữ liệu quý giá, ta sẽ không tiến hành xóa outliers.

## Feature encoding

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
categorical_columns = df_train.select_dtypes(include=["object"]).columns
for col in categorical_columns:
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])
    
# df_train.info()
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,4.189655,9.042040,1,1,3,3,0,...,0.0,3,4,1,0.0,2,2008,8,4,208500
1,2,20,3,4.394449,9.169623,1,1,3,3,0,...,0.0,3,4,1,0.0,5,2007,8,4,181500
2,3,60,3,4.234107,9.328212,1,1,0,3,0,...,0.0,3,4,1,0.0,9,2008,8,4,223500
3,4,70,3,4.110874,9.164401,1,1,0,3,0,...,0.0,3,4,1,0.0,2,2006,8,0,140000
4,5,60,3,4.442651,9.565284,1,1,0,3,0,...,0.0,3,4,1,0.0,12,2008,8,4,250000


## Save file

We will save train and test files to use for the future

In [14]:
df_train = df_train.drop(['Id'], axis=1)
df_test = df_test.drop(['Id'], axis=1)
df_train.to_csv(join(output_dir, "preprocessed_train.csv"), index=False)
df_test.to_csv(join(output_dir, "preprocessed_test.csv"), index=False)
print("Successfully saved preprocessed data.")

Successfully saved preprocessed data.


# The end